In [ ]:
# 킬링 끝음절 리스트 추가(추후에 해도 될듯)
# 비속어, 한글자 입력 시 다시 입력할 수 있게 하기   -> 완료
# 첫 음절만 같게 하고 띄어쓰기 -> 완료
# 특수문자 입력 시 다시 입력
# 끝음절이 자음 또는 모음 하나만 있을 경우 종료

In [ ]:
import requests
import json
import random
import re

# 우리말샘 사전 API
api = 'API 입력력'

# 킬링 끝음절 리스트 지정
ban_last_chars = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌', '섯', '뺌',
                  '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '륨', '슘', '듐', '츔' '썹']
# 비속어 리스트 지정
ban_words = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 사용자가 입력한 단어 리스트
said_y = []
# 비속어, 띄어쓰기 등으로 잘못된 형식의 입력 단어 리스트
said_w = []

# 첫 입력
query = input("단어를 입력하세요(종료를 원하면 '그만'을 입력해주세요.): ")
# 두 글자 이상으로 단어 최소 길이 제한
while len(query) <= 1:
    said_w.append(query)
    query = input("두 글자 이상의 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    
# 비속어 제한
if query in ban_words:
    said_w.append(query)
    query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")

#띄어쓰기 제한
if query.find(' ') != -1:
    said_w.append(query)
    query = input("올바른 단어를 입력해주세요.")

# 사용자가 입력한 단어 리스트에 추가 
said_y.append(query)

while True:
    # 단어 검색 요청
    url = f"https://opendict.korean.go.kr/api/search?certkey_no=6828&key={api}&target_type=search&req_type=json&part=word&q={query[-1]}&sort=date&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
    response = requests.get(url).text
    response = json.loads(response)

    # 이용할 단어 담을 리스트 생성
    word_list = []

    # word_list 리스트에 단어 추가가
    for i in range(100):
        try:
            a = response['channel']['item'][i]['word']
            if a not in said_y:
                word_list.append(a)
        except:
            pass
        
        # 킬링 끝음절, 하이픈(-) 포함 단어 필터링링
        filtered_words = [
            w for w in word_list
            if w[-1] not in ban_last_chars
            and '-' not in w
        ]

    # com이 더이상 이을 수 있는 단어가 없을 때
    if not word_list:
        print('더 이상 이어지는 단어가 없습니다. 당신의 승리!')
        break

    # 필터링 된 단어에서 랜덤으로 추출
    com = random.choice(filtered_words)

    # 사용자 입력 단어와 com의 단어 출력
    print('you:', query, '|', 'com:', com, '\n', '-'*20)

    # com이 사용한 단어 리스트에 추가가
    said_y.append(com)

    # 두 번째 이상의 입력/입력해야할 첫 글자 언급
    query = input(f"{com[-1]}로 시작하는 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    
    # 사용자 의지로 게임 종료
    if query == '그만': 
        print('게임 종료!')
        break
    
    # 잘못된 단어 입력
    if query[0] != com[-1]: 
        print(f'{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!')
        break

    # 중복 입력 금지
    if query in said_y: 
        print(f'{query} 은(는) 이미 입력한 단어입니다. com의 승리!')
        break
    
    # # 잘못된 형식의 단어 2회 이상 입력 시 게임 종료
    # if query in said_w:
    #     print('com의 승리!')
    #     break
    
    # 단어 최소 길이 제한
    if len(query) <= 1: 
        said_w.append(query)
        query = input('두 글자 이상의 단어를 입력해주세요.')
        continue

    # 비속어 제한
    if query in ban_words:  
        said_w.append(query)
        query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")
        continue
    
    # 띄어쓰기 제한
    if query.find(' ') != -1:
        said_w.append(query)
        query = input("올바른 단어를 입력해주세요.")
        print(said_w)
        continue


    # 사용자 입력 단어 리스트에 추가
    said_y.append(query)


you: 고  | com: 부담 
 --------------------
['고 ', '담 ']
you: 담장 | com: 장화 
 --------------------
게임 종료!


In [ ]:
# 전체 코드 리팩터링 + 잘못된 입력 2회 이상 시 게임 종료 조건 추가

import requests
import json
import random

# 우리말샘 사전 API 키 (테스트 환경에서는 'API 입력' 상태 유지)
api = 'AIP 입력'

# 킬링 끝음절 리스트 지정
ban_last_chars = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌', '섯', '뺌',
                  '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '륨', '슘', '듐', '츔', '썹']
# 비속어 리스트 지정
ban_words = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 사용자가 입력한 단어 리스트
said_y = []
# 비속어, 띄어쓰기 등으로 잘못된 형식의 입력 단어 리스트
said_w = []

def check_invalid(query):
    """잘못된 입력 시 said_w에 추가하고 2회 이상이면 True 반환"""
    said_w.append(query)
    if len(said_w) >= 2:
        print('잘못된 입력이 2회 이상입니다. com의 승리!')
        return True
    return False

# 첫 입력
query = input("단어를 입력하세요(종료를 원하면 '그만'을 입력해주세요.): ")
while len(query) <= 1:
    if check_invalid(query) == True: break
    query = input("두 글자 이상의 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
if query in ban_words:
    if check_invalid(query) == True: exit()
    query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")
if ' ' in query:
    if check_invalid(query) == True: exit()
    query = input("올바른 단어를 입력해주세요.")

said_y.append(query)

while True:
    url = f"https://opendict.korean.go.kr/api/search?certkey_no=6828&key={api}&target_type=search&req_type=json&part=word&q={query[-1]}&sort=date&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
    response = requests.get(url).text
    response = json.loads(response)

    word_list = []
    for i in range(100):
        try:
            a = response['channel']['item'][i]['word']
            if a not in said_y:
                word_list.append(a)
        except:
            pass

    filtered_words = [w for w in word_list if w[-1] not in ban_last_chars and '-' not in w]

    if not filtered_words:
        print('더 이상 이어지는 단어가 없습니다. 당신의 승리!')
        break

    com = random.choice(filtered_words)
    print('you:', query, '|', 'com:', com, '\n', '-' * 20)
    said_y.append(com)

    query = input(f"{com[-1]}로 시작하는 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    if query == '그만':
        print('게임 종료!')
        break
    if query[0] != com[-1]:
        print(f'{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!')
        break
    if query in said_y:
        print(f'{query} 은(는) 이미 입력한 단어입니다. com의 승리!')
        break
    if len(query) <= 1:
        if check_invalid(query) == True: break
        query = input('두 글자 이상의 단어를 입력해주세요.')
        continue
    if query in ban_words:
        if check_invalid(query) == True: break
        query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")
        continue
    if ' ' in query:
        if check_invalid(query) == True: break
        query = input("올바른 단어를 입력해주세요.")
        continue

    said_y.append(query)


you: 단  | com: 경보 
 --------------------
잘못된 입력이 2회 이상입니다. com의 승리!


In [ ]:
import requests
import json
import random

# 우리말샘 사전 API 키 (테스트 환경에서는 'API 입력' 상태 유지)
api = 'API 입력'

# 킬링 끝음절 리스트 지정
ban_last_chars = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌', '섯', '뺌',
                  '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '륨', '슘', '듐', '츔', '썹']
ban_words = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']

said_y = []  # 사용자가 입력한 단어 리스트
said_w = []  # 잘못된 형식의 단어 리스트
invalid_streak = 0  # 연속 잘못 입력 횟수

# 잘못된 입력 처리 함수
def check_invalid(query):
    global invalid_streak
    said_w.append(query)
    invalid_streak += 1
    if invalid_streak > 1:
        print('잘못된 입력 연속 2회입니다. com의 승리!')
        return True
    if len(said_w) > 1:
        print('잘못된 입력 누적 2회입니다. com의 승리!')
        return True
    return False

# 첫 입력
query = input("단어를 입력하세요(종료를 원하면 '그만'을 입력해주세요.): ")
while len(query) <= 1:
    if check_invalid(query): exit()
    query = input("두 글자 이상의 단어를 입력해주세요: ")

if query in ban_words:
    if check_invalid(query): exit()
    query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요: ")

if ' ' in query:
    if check_invalid(query): exit()
    query = input("올바른 단어를 입력해주세요: ")

said_y.append(query)
invalid_streak = 0

while True:
    url = f"https://opendict.korean.go.kr/api/search?certkey_no=6828&key={api}&target_type=search&req_type=json&part=word&q={query[-1]}&sort=date&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
    response = requests.get(url).text
    response = json.loads(response)

    word_list = []
    for i in range(100):
        try:
            a = response['channel']['item'][i]['word']
            if a not in said_y:
                word_list.append(a)
        except:
            pass

    filtered_words = [w for w in word_list if w[-1] not in ban_last_chars and '-' not in w]

    if not filtered_words:
        print('더 이상 이어지는 단어가 없습니다. 당신의 승리!')
        break

    com = random.choice(filtered_words)
    print('you:', query, '|', 'com:', com, '\n', '-' * 20)
    said_y.append(com)

    query = input(f"{com[-1]}로 시작하는 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")

    if query == '그만':
        print('게임 종료!')
        break

    if query[0] != com[-1]:
        print(f'{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!')
        break

    if query in said_y:
        print(f'{query} 은(는) 이미 입력한 단어입니다. com의 승리!')
        break

    if len(query) <= 1:
        if check_invalid(query): break
        query = input('두 글자 이상의 단어를 입력해주세요: ')
        continue

    if query in ban_words:
        if check_invalid(query): break
        query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요: ")
        continue

    if ' ' in query:
        if check_invalid(query): break
        query = input("올바른 단어를 입력해주세요: ")
        continue

    said_y.append(query)
    invalid_streak = 0


잘못된 입력 연속 2회입니다. com의 승리!
잘못된 입력 연속 2회입니다. com의 승리!
잘못된 입력 연속 2회입니다. com의 승리!
you: 그만 | com: 만찬 
 --------------------
게임 종료!


: 

In [ ]:
# 개선된 구조: 입력 시 2회 누적 잘못된 입력이 있어야 종료되도록 수정

import requests
import json
import random

API_KEY = 'API 입력'

BAN_LAST_CHARS = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌',
                  '섯', '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '슘', '듐', '츔', '썹']
BAN_WORDS = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']

said_y = []

def is_invalid(word):
    return len(word) <= 1 or word in BAN_WORDS or ' ' in word

invalid_count = 0

def get_valid_input(prompt):
    """올바른 단어 입력 받기. 잘못된 입력이 2회 이상이면 None 반환"""
    global invalid_count
    while True:
        word = input(prompt).strip()
        if word == '그만':
            return word
        if is_invalid(word):
            invalid_count += 1
            if invalid_count >= 2:
                print("잘못된 입력이 2회 이상입니다. com의 승리!")
                return None
        else:
            return word

def get_next_word(last_char):
    try:
        url = (
            f"https://opendict.korean.go.kr/api/search?"
            f"certkey_no=6828&key={API_KEY}&target_type=search"
            f"&req_type=json&part=word&q={last_char}&sort=date"
            f"&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
        )
        response = requests.get(url).text
        data = json.loads(response)

        word_list = []
        for item in data.get("channel", {}).get("item", []):
            word = item.get("word")
            if word and word not in said_y and '-' not in word and word[-1] not in BAN_LAST_CHARS:
                word_list.append(word)

        return word_list
    except Exception:
        print("API 요청 오류가 발생했습니다. API 키를 확인해주세요.")
        return []

def main():
    query = get_valid_input("단어를 입력하세요(종료는 '그만'): ")
    if query is None or query == '그만':
        return
    said_y.append(query)

    while True:
        next_words = get_next_word(query[-1])
        if not next_words:
            print("더 이상 이어지는 단어가 없습니다. 당신의 승리!")
            break

        com = random.choice(next_words)
        print(f"you: {query} | com: {com}\n" + "-" * 20)
        said_y.append(com)

        query = get_valid_input(f"{com[-1]}로 시작하는 단어를 입력하세요(종료는 '그만'): ")
        if query is None or query == '그만':
            print("게임 종료!")
            break
        if query[0] != com[-1]:
            print(f"{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!")
            break
        if query in said_y:
            print(f"{query} 은(는) 이미 입력한 단어입니다. com의 승리!")
            break

        said_y.append(query)

# input()이 이 환경에서는 동작하지 않으므로 실행은 막아둠
main()


you: '미 | com: 미아
--------------------
게임 종료!
